In [87]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import tensorflow as tf
import matplotlib
import os
import matplotlib.pyplot as plt
from datetime import datetime
tf.set_random_seed(77)

mockForecastDictionary = {}
realForecastDictionary = {}

def LearningModuleRunner(rawArrayDatas, processId, forecastDay,dayOrWeekOrMonth):
    #TODO make dayOrWeekOrMonth parameter
    dayOrWeekOrMonth=dayOrWeekOrMonth
    # options:
    # 'day', 'week', 'month'

    feature = 'DayOfWeek_WeekNumber_Month_Season'
    # options:
    # dayOrWeekOrMonth='day': 'DayOfWeek_WeekNumber_Month_Season','DayOfWeek01_WeekNumber_Month_Season'//
    # dayOrWeekOrMonth='week': 'WeekNumber_Month_Season_Year'

#     LoggingManager.PrintLogMessage("LearningManager", "LearningModuleRunner", "start of learning #" + str(processId), DefineManager.LOG_LEVEL_INFO)

    global mockForecastDictionary
    global realForecastDictionary
    mockForcastDay=2*forecastDay

    ##Make txsForRealForecastLstm   [:]
    ds = rawArrayDatas[0]
    y = list(rawArrayDatas[1])
    sales = list(zip(ds, y))
    txsForRealForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastLstm [:-forecastDay]
    ds = rawArrayDatas[0][:-forecastDay]
    y= list(rawArrayDatas[1][:-forecastDay] )
    sales = list(zip(ds, y))
    txsForMockForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForRealForecastBayesian [:-forecastDay] & np.log
    ds = rawArrayDatas[0][:-forecastDay]
    # TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y = list(np.log(rawArrayDatas[1][:-forecastDay]))
    sales = list(zip(ds, y))
    txsForRealForecastBayesian = pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastBayseian   [:-3*forecastDay] & np.log
    ds = rawArrayDatas[0][:-3*forecastDay]
    #TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y= list(np.log(rawArrayDatas[1][:-3*forecastDay]))
    sales = list(zip(ds, y))
    txsForMockForecastBayseian =pd.DataFrame(data=sales, columns=['ds', 'y'])

    #testY for algorithm compare has size of 2*forecastDay:  rawArrayDatas[1][-3*forecastDay:-forecastDay]
    testY= rawArrayDatas[1][-3*forecastDay:-forecastDay]


    if dayOrWeekOrMonth is 'day':
        ####LSTM_day

        #select feature module
        feature='DayOfWeek_WeekNumber_Month_Season'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay,feature)

      
        ####Bayseian_day

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'day')

        #알고리즘 비교
        nameOfBestAlgorithm= AlgorithmCompare(testY)
        ####더 좋은 알고리즘 호출
        if nameOfBestAlgorithm is 'LSTM':
            tf.reset_default_graph()
            realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)

        elif nameOfBestAlgorithm is 'Bayseian':
            realForecastDictionary['Bayseian']=Bayseian(txsForRealForecastBayesian,forecastDay,'day')

            
    elif dayOrWeekOrMonth is 'week':
        
        ####LSTM_week

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)


        ####Bayseian_week

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'week')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        ####더 좋은 알고리즘 호출
        if nameOfBestAlgorithm is 'LSTM':
            tf.reset_default_graph()
            realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)

        elif nameOfBestAlgorithm is 'Bayseian':
     
            realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'week')

            
    elif dayOrWeekOrMonth is 'month':
       
        ####LSTM_month

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)    

        ####Bayseian_month

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'month')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        ####더 좋은 알고리즘 호출
        if nameOfBestAlgorithm is 'LSTM':
            tf.reset_default_graph()
            realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)

        elif nameOfBestAlgorithm is 'Bayseian':
            realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'month')

        ####################################################################################BAYSEIAN
    # tf.reset_default_graph()
    # realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay, feature)

    data = rawArrayDatas[1][:-forecastDay] + realForecastDictionary[nameOfBestAlgorithm]
    date= rawArrayDatas[0]
    print("===================================THE END===================================================")
    return realForecastDictionary[nameOfBestAlgorithm]


def LSTM(txs, forecastDay, features):
    tf.reset_default_graph()
    tf.set_random_seed(77)
    # Add basic date related features to the table
    year = lambda x: datetime.strptime(x, "%Y-%m-%d").year
    dayOfWeek = lambda x: datetime.strptime(x, "%Y-%m-%d").weekday()
    month = lambda x: datetime.strptime(x, "%Y-%m-%d").month
    weekNumber = lambda x: datetime.strptime(x, "%Y-%m-%d").strftime('%V')
    txs['year'] = txs['ds'].map(year)
    txs['month'] = txs['ds'].map(month)
    txs['weekNumber'] = txs['ds'].map(weekNumber)
    txs['dayOfWeek'] = txs['ds'].map(dayOfWeek)

    # Add non-basic date related features to the table
    seasons = [0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0]  # dec - feb is winter, then spring, summer, fall etc
    season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d").month - 1)]
    day_of_week01s = [0, 0, 0, 0, 0, 1, 1]
    day_of_week01 = lambda x: day_of_week01s[(datetime.strptime(x, "%Y-%m-%d").weekday())]
    txs['season'] = txs['ds'].map(season)
    txs['dayOfWeek01'] = txs['ds'].map(day_of_week01)

    # Backup originalSales
    originalSales = list(txs['y'])
    sales = list(txs['y'])
    
    #week number는 경계부분에서 약간 오류가 있다.
    if features is 'DayOfWeek_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]
    elif features is 'DayOfWeek01_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek01']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]

    elif features is 'WeekNumber_Month_Season_Year':
        tempxy = [list(txs['weekNumber']), list(txs['month']), list(txs['season']), list(txs['year']), sales]
    
   
    xy = np.array(tempxy).transpose().astype(np.float)
    


    # Backup originalXY for denormalize
    originalXY = np.array(tempxy).transpose().astype(np.float)
    xy = minMaxNormalizer(xy)

    # TRAIN PARAMETERS
    # data_dim은 y값 도출을 위한 feature 가지수+1(독립변수 가지수 +1(y포함))
    data_dim = 5
    # data_dim크기의 data 한 묶음이 seq_length만큼 input으로 들어가
    seq_length = 10
    # output_dim(=forecastDays)만큼의 다음날 y_data를 예측
    output_dim = forecastDay
    # hidden_dim은 정말 임의로 설정
    hidden_dim = 100
    # learning rate은 배우는 속도(너무 크지도, 작지도 않게 설정)
    learning_rate = 0.001
    iterations=1000
    # Build a series dataset(seq_length에 해당하는 전날 X와 다음 forecastDays에 해당하는 Y)
    x = xy
    y = xy[:, [-1]]
    dataX = []
    dataY = []
    for i in range(0, len(y) - seq_length - forecastDay + 1):
        _x = x[i:i + seq_length]
        _y = y[i + seq_length:i + seq_length + forecastDay]
        _y = np.reshape(_y, (forecastDay))
        dataX.append(_x)
        dataY.append(_y)

    train_size = int(len(dataY) - forecastDay)
    # train_size = int(len(dataY) * 0.7)
    test_size = len(dataY) - train_size

    trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:])

    trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:])

    X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
    Y = tf.placeholder(tf.float32, [None, forecastDay])

    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
    outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=None)
    loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)

    denormalizedTestY = originalSales[train_size + seq_length:]
    #     denormalizedTestY_feed=np.array([[i] for i in denormalizedTestY])

    targets = tf.placeholder(tf.float32, [None, 1])
    predictions = tf.placeholder(tf.float32, [None, 1])

    count = 0
    with tf.Session() as sess:

        # 초기화
        init = tf.global_variables_initializer()
        sess.run(init)

        # Training step
        for i in range(iterations):

            _, step_loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
            print("[step: {}] loss: {}".format(i, step_loss))


        # Test step
        test_predict = minMaxDeNormalizer(sess.run(Y_pred, feed_dict={X: testX}), originalXY)
        realSale = minMaxDeNormalizer(testY[-1], originalXY)
    return test_predict[-1].tolist()

def Bayseian(txs, forecastDay, unit):
    global mockForecastDictionary
    global realForecastDictionary

    if unit is 'day':
        if (len(txs) < 366):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)


    elif unit is 'week':
        if(len(txs)<53):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

    elif unit is 'month':
        if(len(txs)<12):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)


    #date = [d.strftime('%Y-%m-%d') for d in forecastProphetTable['ds']]
    return [np.exp(y) for y in forecastProphetTable['yhat'][-forecastDay:]]

def rmse(a,b):
    sum=0
    for i in range(len(a)):
        sum=sum+(a[i]-b[i])**2
    return np.sqrt(sum/len(a))

def minMaxNormalizer(data):
    numerator=data-np.min(data)
    denominator=np.max(data)-np.min(data)
    return numerator/(denominator+1e-7)

def minMaxDeNormalizer(data, originalData):
    shift=np.min(originalData)
    multiplier=np.max(originalData)-np.min(originalData)
    return (data+shift)*multiplier

def AlgorithmCompare(testY):
    global mockForecastDictionary
    nameOfBestAlgorithm = 'LSTM'
    minData = rmse(testY, mockForecastDictionary[nameOfBestAlgorithm])
    rms = 0
    for algorithm in mockForecastDictionary.keys():
        rms = rmse(testY, mockForecastDictionary[algorithm])
        if rms < minData:
            nameOfBestAlgorithm = algorithm
    print('testY is: ', testY)
    print('\n')
    print('LSTM forecast :',mockForecastDictionary['LSTM'] , '\n@@@@@LSTM rmse: ',rmse(testY, mockForecastDictionary['LSTM']) )
    print('Bayseian forecast :',mockForecastDictionary['Bayseian'], '\n@@@@@Bayseian rmse: ',rmse(testY, mockForecastDictionary['Bayseian']) ) 
    print('\n')
    print(nameOfBestAlgorithm, 'WON!!!!!!')
    return nameOfBestAlgorithm



In [88]:
# def LSTM(txs, forecastDay, features)
# def Bayseian(txs, forecastDay, unit)

In [126]:
columns=['ds','y']
df=pd.read_table('webMonth.csv', sep=',',header=None,names=columns )
forecastDay=int(len(df)*0.1)
print(len(df), forecastDay)


98 9


In [128]:
# df['y'][-forecastDay:]=[1]*forecastDay
rawArrayDatas=[[i for i in df['ds']],[i for i in df['y']]]

In [135]:
rawArrayDatas[1]

[4958.0,
 8359.2999999999993,
 12178.14286,
 1735.5357140000001,
 2178.5666670000001,
 1424.419355,
 1237.7142859999999,
 1343.4545449999998,
 2100.580645,
 3135.5999999999999,
 2644.517241,
 2929.0,
 2971.4838709999999,
 4590.8709680000002,
 2453.4642859999999,
 1603.2903229999999,
 1760.5,
 1483.4838710000001,
 1376.5333330000001,
 1725.9354840000001,
 2844.6451609999999,
 5786.4615380000005,
 6445.6428569999998,
 6952.2142860000004,
 7556.0322580000002,
 13760.172409999999,
 12469.23077,
 2277.3225809999999,
 2503.9666670000001,
 1698.451613,
 1425.5357140000001,
 1848.6153850000001,
 3838.3225810000004,
 9605.4666670000006,
 6985.6774189999996,
 8732.5,
 7899.0645159999995,
 9540.9032260000004,
 3907.1071430000002,
 2085.8709680000002,
 3015.5666670000001,
 1886.193548,
 1695.9000000000001,
 3470.6451609999999,
 4622.2258060000004,
 9086.6896550000001,
 6864.8666670000002,
 7461.9666670000006,
 7556.3103449999999,
 19610.45161,
 25008.793099999999,
 24771.03226,
 3943.3103450000003

In [129]:
# columns=['ds','y']
# txs=pd.read_table('walMonth_train.csv', sep=',',header=None,names=columns )

In [130]:
# len(txs)
# LSTM(txs, 7, features)
# Bayseian(txs, 7, 'month')

In [137]:
answer=LearningModuleRunner(rawArrayDatas,7,9,'month') #7은 processId

[step: 0] loss: 85.34269714355469
[step: 1] loss: 82.27372741699219
[step: 2] loss: 79.27803039550781
[step: 3] loss: 76.28092193603516
[step: 4] loss: 73.20899963378906
[step: 5] loss: 70.00135040283203
[step: 6] loss: 66.61135864257812
[step: 7] loss: 63.006370544433594
[step: 8] loss: 59.174720764160156
[step: 9] loss: 55.14341735839844
[step: 10] loss: 51.01003646850586
[step: 11] loss: 46.99449157714844
[step: 12] loss: 43.51350402832031
[step: 13] loss: 41.235565185546875
[step: 14] loss: 40.80485916137695
[step: 15] loss: 41.739646911621094
[step: 16] loss: 42.238494873046875
[step: 17] loss: 41.29499053955078
[step: 18] loss: 39.41954803466797
[step: 19] loss: 37.58209991455078
[step: 20] loss: 36.416778564453125
[step: 21] loss: 36.05183410644531
[step: 22] loss: 36.273292541503906
[step: 23] loss: 36.75611114501953
[step: 24] loss: 37.224727630615234
[step: 25] loss: 37.5161247253418
[step: 26] loss: 37.57550048828125
[step: 27] loss: 37.42603302001953
[step: 28] loss: 37.137

[step: 240] loss: 24.12952995300293
[step: 241] loss: 24.08083724975586
[step: 242] loss: 24.058242797851562
[step: 243] loss: 24.079835891723633
[step: 244] loss: 24.097179412841797
[step: 245] loss: 23.99040985107422
[step: 246] loss: 23.753414154052734
[step: 247] loss: 23.645673751831055
[step: 248] loss: 23.699792861938477
[step: 249] loss: 23.67043685913086
[step: 250] loss: 23.49062728881836
[step: 251] loss: 23.386672973632812
[step: 252] loss: 23.410259246826172
[step: 253] loss: 23.362802505493164
[step: 254] loss: 23.214773178100586
[step: 255] loss: 23.135652542114258
[step: 256] loss: 23.13543701171875
[step: 257] loss: 23.07528305053711
[step: 258] loss: 22.950536727905273
[step: 259] loss: 22.879436492919922
[step: 260] loss: 22.859811782836914
[step: 261] loss: 22.79844093322754
[step: 262] loss: 22.69306755065918
[step: 263] loss: 22.611509323120117
[step: 264] loss: 22.570812225341797
[step: 265] loss: 22.524248123168945
[step: 266] loss: 22.44049835205078
[step: 267]

[step: 477] loss: 10.502227783203125
[step: 478] loss: 10.352712631225586
[step: 479] loss: 10.234315872192383
[step: 480] loss: 10.209009170532227
[step: 481] loss: 10.247021675109863
[step: 482] loss: 10.278398513793945
[step: 483] loss: 10.26546573638916
[step: 484] loss: 10.194235801696777
[step: 485] loss: 10.100150108337402
[step: 486] loss: 10.006471633911133
[step: 487] loss: 9.938323974609375
[step: 488] loss: 9.898298263549805
[step: 489] loss: 9.879769325256348
[step: 490] loss: 9.879962921142578
[step: 491] loss: 9.903888702392578
[step: 492] loss: 9.976554870605469
[step: 493] loss: 10.122981071472168
[step: 494] loss: 10.430543899536133
[step: 495] loss: 10.79255485534668
[step: 496] loss: 11.127605438232422
[step: 497] loss: 10.701409339904785
[step: 498] loss: 9.928866386413574
[step: 499] loss: 9.529972076416016
[step: 500] loss: 9.849979400634766
[step: 501] loss: 10.203743934631348
[step: 502] loss: 9.883443832397461
[step: 503] loss: 9.426491737365723
[step: 504] lo

[step: 744] loss: 3.841986656188965
[step: 745] loss: 4.109197616577148
[step: 746] loss: 4.574816703796387
[step: 747] loss: 4.553421974182129
[step: 748] loss: 4.382021903991699
[step: 749] loss: 3.6426637172698975
[step: 750] loss: 3.208861827850342
[step: 751] loss: 3.292285919189453
[step: 752] loss: 3.624763011932373
[step: 753] loss: 3.7979660034179688
[step: 754] loss: 3.476973056793213
[step: 755] loss: 3.1601381301879883
[step: 756] loss: 3.1614108085632324
[step: 757] loss: 3.364987373352051
[step: 758] loss: 3.4487991333007812
[step: 759] loss: 3.248363971710205
[step: 760] loss: 3.05912184715271
[step: 761] loss: 3.0713601112365723
[step: 762] loss: 3.1970772743225098
[step: 763] loss: 3.244366407394409
[step: 764] loss: 3.1301581859588623
[step: 765] loss: 3.0009806156158447
[step: 766] loss: 2.96488618850708
[step: 767] loss: 3.0151898860931396
[step: 768] loss: 3.0683434009552
[step: 769] loss: 3.055471181869507
[step: 770] loss: 2.997745990753174
[step: 771] loss: 2.92

[step: 978] loss: 1.2732657194137573
[step: 979] loss: 1.5205422639846802
[step: 980] loss: 1.2917784452438354
[step: 981] loss: 1.2201286554336548
[step: 982] loss: 1.3870527744293213
[step: 983] loss: 1.2495050430297852
[step: 984] loss: 1.1760594844818115
[step: 985] loss: 1.3142738342285156
[step: 986] loss: 1.210917592048645
[step: 987] loss: 1.133631944656372
[step: 988] loss: 1.242073893547058
[step: 989] loss: 1.1909838914871216
[step: 990] loss: 1.1128592491149902
[step: 991] loss: 1.1808582544326782
[step: 992] loss: 1.1682524681091309
[step: 993] loss: 1.0912456512451172
[step: 994] loss: 1.1353427171707153
[step: 995] loss: 1.144270420074463
[step: 996] loss: 1.0814391374588013
[step: 997] loss: 1.0939284563064575
[step: 998] loss: 1.1154534816741943
[step: 999] loss: 1.071496605873108
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
testY is:  [7608.3999999999996, 8706.1935480000011, 26366.56667, 24504.89286, 2169.0322579999997, 1879

In [138]:
len(answer)

9

In [139]:
answer

[2178.676251601592,
 1461.6115387298612,
 2651.7794099982207,
 3007.6696959836563,
 5874.4991051425168,
 3859.3472809028963,
 3301.8477645126191,
 4105.2784993045379,
 11852.520793829079]